In [11]:
# pip install -r requirements.txt

In [12]:
# Imports
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
from datetime import date, timedelta

# Dynamic timeline (end is exclusive, so add a day to grab latest close)
start_date = date(2015, 1, 1)
end_date = date.today() + timedelta(days=1)


In [13]:
# Cross-sector mix for future modeling
tickers = ["AAPL","MSFT","AMZN","GOOGL","NVDA","TSLA","JPM","WMT",
           "DAL","UAL","LMT","RTX","NOC","XOM"]

print(f"{len(tickers)} tickers")

14 tickers


In [14]:
# Download price data
data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)

[*********************100%***********************]  14 of 14 completed


In [15]:
# Flatten multi-index into one row per ticker and date
data_flat = data.stack(level=1, future_stack=True).reset_index()

data_flat.rename(columns={
    "level_1": "Ticker",
    "Adj Close": "AdjClose",
    "Close": "Close",
    "Open": "Open",
    "High": "High",
    "Low": "Low",
    "Volume": "Volume"
}, inplace=True)

data_flat.head()

Price,Date,Ticker,Close,High,Low,Open,Volume
0,2015-01-02,AAPL,24.237553,24.705322,23.798602,24.694237,212818400
1,2015-01-02,AMZN,15.426000,15.737500,15.348000,15.629000,55664000
2,2015-01-02,DAL,43.064999,43.791795,42.653437,43.712986,8637300
3,2015-01-02,GOOGL,26.278944,26.589101,26.196068,26.430299,26480000
4,2015-01-02,JPM,46.720932,47.072328,46.406916,46.489158,12600000


In [16]:
# Feature engineering: technical indicators
data_flat["Return"] = data_flat.groupby("Ticker")["Close"].pct_change()

data_flat["RollingVol"] = (
    data_flat.groupby("Ticker")["Return"]
    .rolling(window=10)
    .std()
    .reset_index(0, drop=True)
)

data_flat["RSI"] = data_flat.groupby("Ticker")["Close"].transform(
    lambda x: ta.momentum.rsi(x, window=14)
)

# SMA ratios instead of absolute levels to avoid price scale leakage
sma_20 = data_flat.groupby("Ticker")["Close"].transform(
    lambda x: x.rolling(20).mean()
)
sma_50 = data_flat.groupby("Ticker")["Close"].transform(
    lambda x: x.rolling(50).mean()
)

data_flat["Price_to_SMA20"] = data_flat["Close"] / sma_20
data_flat["Price_to_SMA50"] = data_flat["Close"] / sma_50
data_flat["SMA20_to_SMA50"] = sma_20 / sma_50

data_flat["Volume_Z"] = data_flat.groupby("Ticker")["Volume"].transform(
    lambda x: (x - x.rolling(20).mean()) / x.rolling(20).std()
)

In [17]:
# Merge VIX index (leakage-safe forward-fill)
macro = yf.download(
    ["^VIX"],
    start=start_date,
    end=end_date,
    auto_adjust=True
)

macro.columns = [col[0] for col in macro.columns]
macro = macro.reset_index().rename(columns={"Close": "VIX"})

# Forward-fill VIX BEFORE merge to avoid cross-ticker leakage
macro = macro.sort_values("Date")
macro["VIX"] = macro["VIX"].ffill()

# Merge VIX by date
data_flat = data_flat.merge(macro[["Date", "VIX"]], on="Date", how="left")

# Sanity check: leading NaNs (before VIX data starts)
print(f"Leading VIX NaNs: {data_flat['VIX'].isna().sum()}")

[*********************100%***********************]  1 of 1 completed


Leading VIX NaNs: 0


In [ ]:
# Sort by Ticker and Date to ensure proper time ordering
data_flat = data_flat.sort_values(["Ticker", "Date"]).reset_index(drop=True)

# Lagged_Return: prior day's return (t-1) for predicting t+1 outcomes
data_flat["Lagged_Return"] = data_flat.groupby("Ticker")["Return"].shift(1)


# Create targets (t → t+1 labels, leakage-safe)

# NextReturn: tomorrow's return (continuous target)
data_flat["NextReturn"] = data_flat.groupby("Ticker")["Return"].shift(-1)

# NextDirection: next-day return direction (binary classification target)
data_flat["NextDirection"] = np.where(
    data_flat["NextReturn"].isna(),
    np.nan,
    (data_flat["NextReturn"] > 0).astype(float)
)

# NextVolSpike: next-day volatility spike (binary classification target)
# NextVolSpike(t) = 1 if RollingVol at t+1 exceeds 80th percentile of
# historical RollingVol computed using ONLY data up to t-1

next_day_vol = data_flat.groupby("Ticker")["RollingVol"].shift(-1)

# Expanding threshold uses ONLY information available up to t-1
# shift(1) ensures we don't include today's RollingVol in threshold
expanding_threshold = data_flat.groupby("Ticker")["RollingVol"].transform(
    lambda s: s.shift(1).expanding(min_periods=126).quantile(0.8)
)

data_flat["NextVolSpike"] = np.where(
    next_day_vol.isna() | expanding_threshold.isna(),
    np.nan,
    (next_day_vol > expanding_threshold).astype(float)
)

# Sanity check: NaN counts per target
print("Target NaN counts (from shifting/warm-up windows):")
print(f"  Lagged_Return: {data_flat['Lagged_Return'].isna().sum()} NaNs")
print(f"  NextReturn:    {data_flat['NextReturn'].isna().sum()} NaNs")
print(f"  NextDirection: {data_flat['NextDirection'].isna().sum()} NaNs")
print(f"  NextVolSpike:  {data_flat['NextVolSpike'].isna().sum()} NaNs")
print(f"\nExpected NaNs:")
print(f"  Lagged_Return: 14 (1 per ticker from shift)")
print(f"  NextReturn & NextDirection: 14 (1 per ticker from shift)")
print(f"  NextVolSpike: ~1,890 (126-day warm-up + 1 shift per ticker)")

In [19]:
from pathlib import Path
Path('data').mkdir(exist_ok=True)

In [ ]:
# Save datasets
from pathlib import Path

Path("data").mkdir(exist_ok=True)

# Save FULL dataset (includes rows with incomplete features for EDA)
data_flat.to_csv("data/merged_features_full.csv", index=False)

# Drop rows after all feature/label engineering to remove terminal rows
data_flat_clean = data_flat.dropna().copy()

# Convert targets to int (safe after dropna)
data_flat_clean["NextDirection"] = (
    data_flat_clean["NextDirection"].astype(int)
)
data_flat_clean["NextVolSpike"] = data_flat_clean["NextVolSpike"].astype(int)

# Save CLEAN dataset (model-ready)
data_flat_clean.to_csv("data/merged_features_clean.csv", index=False)

# Summary statistics
print(f"Full dataset (with incomplete features): {data_flat.shape[0]} rows")
print(f"Clean dataset (model-ready): {data_flat_clean.shape[0]} rows")
print(f"Rows dropped (incomplete features + NaN targets): "
      f"{data_flat.shape[0] - data_flat_clean.shape[0]}")
print(f"\nClean dataset target stats:")
print(f"  NextDirection: "
      f"{data_flat_clean['NextDirection'].value_counts().to_dict()}")
print(f"  NextVolSpike:  "
      f"{data_flat_clean['NextVolSpike'].value_counts().to_dict()}")

# Verify Lagged_Return is NOT identical to Return
print(f"\nLagged_Return validation:")
print(f"  Correlation(Return, Lagged_Return): "
      f"{data_flat_clean['Return'].corr(data_flat_clean['Lagged_Return']):.6f}")
print(f"  Are they identical? "
      f"{(data_flat_clean['Return'] == data_flat_clean['Lagged_Return']).all()}")

data_flat_clean.head(10)

### Dataset Outputs

Two datasets are saved for different purposes:

1. **`merged_features_full.csv`** - Complete dataset including rows with incomplete features
   - **Used for**: EDA price history, understanding full market timeline
   - **Contains**: All rows from Jan 2015 onwards, including early 2015 with NaN indicators
   - **Rows**: ~38,668

2. **`merged_features_clean.csv`** - Model-ready dataset with complete features only
   - **Use for**: Feature analysis, correlation matrices, modeling, and predictions
   - **Contains**: Only rows where all technical indicators have complete lookback windows
   - **Rows**: ~36,764
   - **Start date**: July 17, 2015, after 50-day MA, 126-day expanding window requirements are satisfied

The separation created here ensures that our EDA can show the complete market picture while modeling uses only reliable, fully-calculated features.